In [22]:
!pip install datasets openai -q


[notice] A new release of pip is available: 23.1 -> 24.3.1
[notice] To update, run: C:\Users\susha\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip


In [23]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

# Load the dataset with the "question-answer" configuration
dataset1 = load_dataset('rag-datasets/rag-mini-wikipedia', 'question-answer')

df1 = pd.DataFrame(dataset1['test'])
# Display the dataset
print(dataset1)

DatasetDict({
    test: Dataset({
        features: ['question', 'answer', 'id'],
        num_rows: 918
    })
})


In [24]:
df1

,question,answer,id
0,Was Abraham Lincoln the sixteenth President of...,yes,0
1,Did Lincoln sign the National Banking Act of 1...,yes,2
2,Did his mother die of pneumonia?,no,4
3,How many long was Lincoln's formal education?,18 months,6
4,When did Lincoln begin his political career?,1832,8
...,...,...,...
913,Was Wilson president of the American Political...,Yes,1710
914,Did he not cast his ballot for John M. Palmer ...,Yes,1711
915,Did Wilson not spend 1914 through the beginnin...,Yes,1712
916,"Was Wilson , a staunch opponent of antisemitis...",Yes,1713


In [25]:
question = df1['question'][917]
print("Question is ",question)
answer  = df1['answer'][917]
print("answer is",answer)

Question is  What happened in 1917?
answer is raised billions through Liberty loans, imposed an income tax, S08_set up the War Industries Board, promoted labor union growth, supervised agriculture and food production through the Lever Act, took over control of the railroads, and suppressed anti-war movements


In [26]:
question1 = df1['question'][3]
print("Question is ",question1)
answer1  = df1['answer'][3]
print("answer is",answer1)

Question is  How many long was Lincoln's formal education?
answer is 18 months


In [28]:
import openai
from openai import OpenAI
import os


client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def answer_question(text, model="gpt-4o-mini", max_tokens=100):

    try:
        response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant specialized in giving the most accurate answer for a specific question asked."},
            {"role": "user", "content": f"\n{text}"}
        ]
        )
        # Extract the summary from the response
        summary = response.choices[0].message.content
        return summary
    except Exception as e:
        return f"Error: {e}"


In [29]:
result = answer_question(text = question)
print(result)

Several significant events occurred in 1917, including:

1. **Russian Revolution**: The February Revolution led to the abdication of Tsar Nicholas II and the end of the Romanov dynasty. This was followed by the October Revolution, in which the Bolsheviks, led by Vladimir Lenin, seized power from the provisional government.

2. **United States Entry into World War I**: The United States declared war on Germany on April 6, 1917, joining the Allied Powers in World War I after years of neutrality.

3. **Balfour Declaration**: On November 2, 1917, the British government issued the Balfour Declaration, expressing support for a "national home for the Jewish people" in Palestine, which became a pivotal moment in Middle Eastern history.

4. **Battle of Passchendaele**: This significant and grueling battle took place during the summer and fall of 1917 as part of the Flanders campaign in World War I, marked by heavy casualties and harsh conditions.

These events had lasting impacts on global poli

In [30]:
from rouge_score import rouge_scorer

def calculate_rouge(reference, candidate):

    # Initialize a ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    
    # Extract and format the scores
    rouge_scores = {
        'ROUGE-1': scores['rouge1'].fmeasure,
        'ROUGE-2': scores['rouge2'].fmeasure,
        'ROUGE-L': scores['rougeL'].fmeasure
    }
    
    return rouge_scores

print(calculate_rouge(reference = answer,candidate = result))

{'ROUGE-1': 0.08823529411764705, 'ROUGE-2': 0.009900990099009901, 'ROUGE-L': 0.06862745098039215}


Calculate rougue score for next question


In [35]:
result1 = answer_question(text = question1)
print("Result is ",result1)
print(calculate_rouge(reference = answer1,candidate = result1))

Result is  Abraham Lincoln had a limited formal education, totaling about 18 months. His schooling was sporadic, as he grew up in rural areas where access to formal education was limited. Most of his learning came from self-education through reading books and engaging in discussions.
{'ROUGE-1': 0.08695652173913045, 'ROUGE-2': 0.04545454545454545, 'ROUGE-L': 0.08695652173913045}


In [32]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(reference, candidate):
    
    # Tokenize the reference and candidate texts
    reference_tokens = [reference.split()]
    candidate_tokens = candidate.split()
    
    # Apply smoothing to handle edge cases
    smoothing_function = SmoothingFunction().method1
    
    # Compute BLEU score
    bleu_score = sentence_bleu(
        reference_tokens,
        candidate_tokens,
        smoothing_function=smoothing_function
    )
    
    return bleu_score
bleu = calculate_bleu(reference = answer, candidate=result)
print("BLEU Score:", bleu)

BLEU Score: 0.0032728749416290007


In [33]:
bleu = calculate_bleu(reference = answer1, candidate=result1)
print("BLEU Score:", bleu)

BLEU Score: 0.008647922020227043


In [ ]:
# from nltk.translate.bleu_score import sentence_bleu
# from nltk import word_tokenize

# def calculate_bleu_score_from_strings(reference_summary_str, generated_summary_str):

#     # Tokenize the strings into lists of words
#     reference_summary = [word_tokenize(sentence) for sentence in reference_summary_str.split('.')] 
#     generated_summary = word_tokenize(generated_summary_str) 

#     return sentence_bleu(reference_summary, generated_summary)

# # Example Usage:
# reference_summary_str = first_answer
# generated_summary_str = result

# bleu_score = calculate_bleu_score_from_strings(reference_summary_str, generated_summary_str)
# print(f"BLEU Score: {bleu_score}")


